In [ ]:
import pandas as pd
import os
import re
import numpy as np
from numpy import log 
from math import sqrt 

!pip install pdfplumber
!pip install unidecode

#Libreria para importar texto de pdf
import pdfplumber

#librerias para texto
import nltk #Natural Language Tool Kit
#importamos tokenizador de texto
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer

#stemmer mediate algoritmo de porter
from nltk.stem import PorterStemmer 

#stemmer de libreria snowball
from nltk import SnowballStemmer 
import string #funciones adicionales para cadenas



#para reemplazo de caracteres especiales latinos
import unidecode 
import unicodedata

#para visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

     |████████████████████████████████| 46 kB 2.7 MB/s 
     |████████████████████████████████| 5.6 MB 5.5 MB/s 
     |████████████████████████████████| 4.3 MB 49.3 MB/s 
     |████████████████████████████████| 139 kB 60.3 MB/s 
     |████████████████████████████████| 3.6 MB 47.0 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.6.0-py3-none-any.whl size=33688 sha256=2c2bef8da1798862188ddb39bb14c691e1baffd30864c585e81ee5fa1fbe87e5
  Stored in directory: /root/.cache/pip/wheels/58/56/fe/2e93d842ffa9ea97746c1ab253d43502ed61c0689361a0224e
Successfully built pdfplumber
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is inc

     |████████████████████████████████| 235 kB 4.1 MB/s 


In [ ]:
#Establecemos directorio
dir=os.chdir('/content/drive/MyDrive/00-data/')


FileNotFoundError: ignored

In [ ]:
#1- CARGAMOS EL CONTENIDO DEL PDF EN UNA LISTA

In [ ]:
#Cargamos pdf en un objeto pdfplumber
pdf = pdfplumber.open('19900521.pdf')
pdf

In [ ]:
# Proceso de carga de pdf
page = pdf.pages[0]
text = page.extract_text()
print(text)

In [ ]:
# Total de paginas del texto
total_pages = len(pdf.pages)
total_pages

In [ ]:
#indexamos contenido separado por paginas

#creamos lista que almacenará el contenido de todo el archivo, segmentado por paginas
lista_archivo1=[]

#ciclo para cada pagina en el rango del numero de paginas del archivo
for pagina in range(total_pages):
    #indexamos contenido de texto de cada pagina en una lista auxiliar
    lista_pagina=pdf.pages[pagina].extract_text()
    
    #almacenamos el contenido de texto de la pagina indexada en la lista final del archivo
    lista_archivo1.append(lista_pagina)

#revisamos contenido de las primeras 5 paginas del documento   
for pagina in range(0,4):
    print('Página ',str(pagina+1),':\n',lista_archivo1[pagina], '\n')
    

In [ ]:
#2- VACIAMOS CONTENIDO DEL PDF EN UN DATAFRAME

In [ ]:
#Pasamos la lista de páginas a un dataframe
df_paginas=pd.DataFrame(lista_archivo1, columns=['contenido_pagina_crudo'])
df_paginas.head(10)

In [ ]:
#3- LIMPE Y NORMALICE EL CONTENIDO MEDIANTE FUNCION AD HOC

In [ ]:
#importamos stopwords
from nltk.corpus import stopwords 

#descargamos stopwords
nltk.download('stopwords')

#importamos tokenizador de texto
from nltk.tokenize import word_tokenize 

#para reemplazo de caracteres especiales latinos
import unidecode 
import unicodedata

#stemmer mediate algoritmo de porter
from nltk.stem import PorterStemmer 

#stemmer de libreria snowball
from nltk import SnowballStemmer 

#creamos lista de stopwords en castellano
stopwords=stopwords.words('spanish')

#normalizamos stopwords, removemos tildes y otros caracteres latinos para el match con texto normalizado
#creamos lista vacía que almacenara palabras normalizadas
stop_words=[]
#ciclo en el que cada palabra
for word in stopwords:
    #es normalizada
    word_norm = unicodedata.normalize('NFD', word).encode('ascii', 'ignore').decode("utf-8")
    
    #y almacenada en la nueva lista
    stop_words.append(word_norm)
    print(word, word_norm)
    

In [ ]:
####LEMATIZACIÓN

#Con la lematización pasa algo similar, cortar palabras puede hacernos perder su riqueza.
snowball = SnowballStemmer(language='spanish')
#instanciamos el lematizador de porter
#porter= PorterStemmer(languaje='spanish')


####NORMALIZACIÓN DE TEXTO

def limpieza(texto):
    #pasamos a minusculas
    texto_norm=texto.lower()
    
    #removemos espacios al final y al inicio de cada cadena
    texto_norm=texto_norm.strip()
    
    #removemos dobles espacios: ojo que si los eliminamos, concatenamos palabras
    texto_norm=re.sub('\s',' ', texto_norm)
    
    #nuevo_texto = unidecode.unidecode(nuevo_texto)
    texto_norm = unicodedata.normalize('NFD', texto_norm).encode('ascii', 'ignore').decode("utf-8")
    
    #retenemos caracteres alfanumericos: ojo que incluimos la ñ
    texto_norm=re.sub('[^A-Za-zñ]+', ' ', texto_norm)
    
    #tokenizamos texto: convertir en una lista una cadena
    texto_norm = word_tokenize(texto_norm)
    
    # Eliminación de terminos de largo 1
    texto_norm = [token for token in texto_norm if len(texto_norm) > 1]
    
    #removemos stopwords
    texto_norm=[word for word in texto_norm if not word in stop_words]
    
    #lematizamos texto con el stemmer de snowbal
    #TAREA OPCIONAL: pruebe el stemmer de porter y compare resultados
    texto_norm=[snowball.stem(word) for word in texto_norm]
    
    #resultado de la funcion
    return(texto_norm)

In [ ]:
#IMPLEMENTACIÓN: la funcion de limpieza y normalizacion
df_paginas['contenido_limpio'] = df_paginas['contenido_pagina_crudo'].apply(lambda x: limpieza(x))

#revisamos cabecera
df_paginas.tail()

In [ ]:
#creamos columna que identifica cada hoja, se tienen en total las 81 hojas del pdf original
df_paginas['pagina']=df_paginas.index+1

#revisamos cabecera
df_paginas.tail()

In [ ]:
#4- GENERACION DE GRAFICOS PARA LEY DE ZIPF Y HEAPS

In [ ]:
#creamos data frame que intersecta a cada página con cada termino
doc_term=df_paginas.explode(column= 'contenido_limpio')

#cambiamos nombre de columna para identificar termino
doc_term=doc_term.rename(columns={'contenido_limpio':'termino'})

#vista de cabecera
doc_term.tail(50)

In [ ]:
#Construimos el vocabulario
#agregamos datos
vocabulario=doc_term.groupby(['termino']).count().sort_values(by='pagina', ascending=False).reset_index()

#cambiamos nombre para identificar frecuencia del termino
vocabulario  = vocabulario.rename(columns={'pagina':'frecuencia'})

#agregamos columna con el ranking
vocabulario['ranking']=vocabulario.index+1

#generamos la frecuencia estimada segun formula de la ley de zipf
vocabulario['frecuencia_estimada']=vocabulario.frecuencia[0]/(vocabulario.ranking**2)

#revisamos cabecera
vocabulario.head(60)

In [ ]:
#Graficamos la Ley de Zipf
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(vocabulario.ranking, vocabulario.frecuencia_estimada);
ax.plot(vocabulario.ranking, vocabulario.frecuencia);

# Agregamos titulo y nombres en los ejes
plt.title('Sesión del Congreso Pleno - Ley de Zipf')
plt.xlabel("i")
plt.ylabel("f(i)")
plt.show()

In [ ]:
#agregamos datos a nivel de articulos para ley de heaps
#tener en cuenta que vamos caracterizar a cada articulo segun el numero de palabras totales
#y segun el numero de palabras distintas
#doc_term.groupby(['termino']).count().sort_values(by='articulo_num', ascending=False).reset_index()
total=pd.DataFrame(doc_term.groupby(['pagina'])['termino'].count())
total=total.rename(columns={'termino':'terminos_totales'})

distint=pd.DataFrame(doc_term.groupby(['pagina'])['termino'].nunique())
distint=distint.rename(columns={'termino':'terminos_distintos'})
vocabulario_art=pd.concat([total,distint], axis=1).sort_values(by='terminos_distintos', ascending=True).reset_index()

#parametros para la ley de Heaps
K=10
beta=0.5
vocabulario_art['V']=K*vocabulario_art.terminos_totales**(beta)
vocabulario_art=vocabulario_art.sort_values(by='V')


In [ ]:
vocabulario_art

In [ ]:
#Graficamos la Ley de Heaps
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(vocabulario_art.terminos_totales, vocabulario_art.V);
ax.plot(vocabulario_art.terminos_totales, vocabulario_art.terminos_distintos);
# Agregamos titulo y nombres en los ejes
plt.title('Sesión del Congreso Pleno - Ley de Heaps')
plt.xlabel("n")
plt.ylabel("V")
plt.show()

In [ ]:
#REPRESENTACION DE TEXTO TOKENIZADO EN MATRIZ TF-IDF

In [ ]:
#instanciamos modelo para vectorizar:
def identity_tokenizer(text):
    return text

tfidf_vect = TfidfVectorizer(tokenizer=identity_tokenizer,
                             lowercase=False,
                            use_idf=True,
                            ngram_range=(1,2))

#ajustamos vectorizador a los datos
tfidf_vect.fit(df_paginas.contenido_limpio)

#implementamos matriz tf-idf
tfidf_data = tfidf_vect.transform(df_paginas.contenido_limpio)

In [ ]:
tfidf_vect.get_feature_names()

In [ ]:
tfidf_df=pd.DataFrame(tfidf_data.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_df.shape

In [ ]:
tfidf_df